In [2]:
from langchain.document_loaders import DirectoryLoader

directory = 'input_dataset'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

13

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))


42


In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
from credentials_hander import OPENAI_API_KEY
embeddings = OpenAIEmbeddings(model_name="ada", openai_api_key= OPENAI_API_KEY)
query_result = embeddings.embed_query("Hello world")
len(query_result)

1024

In [9]:
from langchain.vectorstores import Pinecone
import pinecone
pinecone.init(
    api_key="46b0be1d-d884-49da-a32b-e774dd426b72",
    environment="asia-southeast1-gcp-free"
)
index_name = 'alyarz-pinecone-index'
index = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [17]:
index

In [10]:
def get_similiar_docs(query, k=2, score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query, k=k)
  else:
    similar_docs = index.similarity_search(query, k=k)
  return similar_docs


In [18]:
index

In [14]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from credentials_hander import OPENAI_API_KEY
# model_name = "text-davinci-003"
# model_name = "gpt-3.5-turbo"
model_name = "gpt-3.5-turbo"
llm = OpenAI(model_name=model_name, openai_api_key=OPENAI_API_KEY)

chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer


In [16]:
query = "December 2019, Discount in LBP?"
answer = get_answer(query)
print(answer)

The discount in LBP in December 2019 is 2%.
